<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2023/ex06notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2023 ex06notebookC

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023

----
## 演習課題



### 準備




In [ ]:
# いろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

このセクションでは，この notebook で使うデータの準備をします．
**講義の notebook で使ったのと同じデータですが，簡単のため，いくつかの変数を削除して扱うことにします**．


独立行政法人統計センターが作成している 教育用標準データセット(SSDSE) の中の「SSDSE-家計消費」というデータを使います．

「SSDSE-家計消費」は，47都道府県の県庁所在市で調査された家計消費に関する226項目の数値から成るデータです．「二人以上の世帯の1世帯あたりの焼き肉（外食）への年間支出金額」みたいな数値が含まれています．

データの詳細はこちら: https://www.nstac.go.jp/sys/files/kaisetsu-C-2022.pdf

このセクションのコードの作成にあたっては，データの前処理の部分でこちらを参考にさせていただきました： https://studiolab.sagemaker.aws/import/github/icoxfog417/mlnote-note/blob/main/notebooks/chapter10_answer.ipynb

以下のコードを上から順に実行しましょう

In [ ]:
##### CSV ファイルを読み込む #####
URL = 'https://www.nstac.go.jp/sys/files/SSDSE-C-2022.csv'
df_raw = pd.read_csv(URL, encoding='cp932') # 文字コードが UTF8 ではなく ShiftJIS なので，encoding オプションを指定して変換
df_raw.head(10) # 最初の10行を表示

In [ ]:
#####  データの前処理 step1  #####

# df_raw の見出し行に記されたコードと，1行目に記されたその項目名との対応表を作り，項目名の行を削除
code = df_raw.iloc[0, :]
print('##### コードと項目名の対応')
print(code)
df1 = df_raw.drop(0, axis=0)

# 'SSDSE-C-2022'列の都市コードと'Prefecture'列の都道府県名，'City'列の県庁所在地名を整理
city = df1.iloc[:, :3]
print()
print('##### 都市コード，都道府県名，県庁所在地名')
print(city.head(5))
df1 = df1.drop(['Prefecture', 'City'], axis=1)
df1 = df1.set_index('SSDSE-C-2022')

df1 = df1.drop('R00000', axis=0) # 'R00000'列は「全国」のデータなのでこれを削除
df1 = df1.astype('float32') # 値を浮動小数点型に変換

print()
df1.head(5)

In [ ]:
#####  データの前処理 step2  #####

# 一人あたりの外食支出金額のデータを作成
df2 = pd.DataFrame()

# コードが 'LB12' で始まる列が，様々な品目の外食支出額．ただし，1世帯あたりの金額
columns = {}
for c in df1.columns:
    if c.startswith('LB12') and c != 'LB12':
        #print(c, code[c])
        columns[c] = code[c]
        df2[c] = df1[c]

# 列名をコードから項目名に変更
df2 = df2.rename(columns=columns)

# 都道府県ごとの平均世帯人数で割って一人あたりの金額にする
df2 = df2.divide(df1['LA03'], axis=0)

# 県庁所在地名の列を追加
df2 = pd.merge(city.loc[:, ['SSDSE-C-2022', 'City']], df2, left_on='SSDSE-C-2022', right_index=True)
df2 = df2.reset_index(drop=True)

df_feature = df2

# DataFrame 表示時の小数部の表示桁数
pd.options.display.precision = 1

# いくつかの列を削除
df_feature.drop(columns=['他の麺類外食', '他の主食的外食', '学校給食', '飲酒代'], inplace=True)

# 最初の5件
df_feature.head(5)

### 問題1


(1) 次のコードを実行すると，`Xorg` にデータの値が格納されます．コードを追加して，`N` と `D` という変数に，このデータの数と次元数をそれぞれ代入して print させましょう．


In [ ]:
Xorg = df_feature.drop(columns=['SSDSE-C-2022', 'City']).to_numpy()



(2) 次のセルに，`Xorg` を標準化したデータに主成分分析を行い，2次元に次元削減するコードを書きましょう．
ただし，いくつかの変数について，次のように名前を付けてください．
- `X`: 標準化したデータを格納した shape が (N, D) の配列
- `Vx`: `X` の分散共分散行列
- `eval`: `Vx` の固有値
- `U`: `Vx` の固有ベクトル．値の並べ方はこれまでと同様に
- `Y`: `X` を2次元に変換したデータを格納した shape が (N, 2) の配列

(3) 次のセルに，`eval` を用いて寄与率と累積寄与率を求めて print するコードを書きましょう．その結果を見て，次の問の答えを考え，メモしておきなさい．
- 元のデータの「変数ごとの分散の和」の 80% 以上を表すためには，削減後の次元数をいくつ以上にするべきか
- 同じく 90% 以上を表すためには（以下同文）

### 問題2

(1) 問題1が正しくできていれば，次のコードを実行することで，横軸に第1主成分，縦軸に第2主成分をとった散布図が描かれるはずです．（散布図の考察はあとにして）実行できることを確認しましょう．

ちなみに，0から数えて1番目から6番目までが東北6県，24番目から28番目までが近畿5府県です．

In [ ]:
# Y の散布図
fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(Y[:, 0], Y[:, 1])
ax.scatter(Y[24:29, 0], Y[24:29, 1], label='kinki5')
ax.scatter(Y[ 1: 7, 0], Y[ 1: 7, 1], label='tohoku6')
ax.set_xlim(-4.5, 4.5)
ax.set_ylim(-4.5, 4.5)
ax.set_aspect('equal')
ax.axhline(0, color='gray')
ax.axvline(0, color='gray')
ax.legend()
for n in range(len(Y)):
    plt.annotate(f'{n}', (Y[n, 0]+0.1, Y[n, 1]+0.1))
plt.show()

(2) 次のセルに，最大固有値に対応する固有ベクトルの値を print するコードを書きなさい．

その値を見て，散布図の右の方に描かれた市／左の方に描かれた市はどのような外食傾向にあるか考えなさい．メモしておくこと．

次のセルを実行すると，22番目と41番目の市の元データを表示させられます．上で考えた通りになっているかどうか確かめましょう．

In [ ]:
df_feature.iloc[[22, 41], :]

(3) 次のセルに，2番目に大きな固有値に対応する固有ベクトルの値を print するコードを書きなさい．

値の正負や大きさを考えて，散布図の上の方に描かれた市／下の方に描かれた市はどのような外食傾向にあるか考えなさい．メモしておくこと．

In [ ]:
df_feature.iloc[[27, 5], :]

(4) 上記の結果と散布図上の位置から，東北6県と近畿5府県のおおまかな外食傾向を考察してメモしておきましょう．お金たくさん使う？比較的何をよく食べる？,etc.

うどん県高松市

In [ ]:
# うどん県高松市
df_feature.iloc[[36], :]